<a href="https://colab.research.google.com/github/yoga3311/Fitness_Trainer_Chatbot/blob/main/Fitness_Trainer_Chatbot3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [9]:
import gradio as gr
import nltk
import random
import string
import tensorflow as tf
from tensorflow import keras
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

nltk.download('punkt')
nltk.download('wordnet')

user_data = {}
user_questions = ["", "Enter your email:", "Enter your age:", "Enter your height (cm):", "Enter your weight (kg):"]
user_keys = ["name", "email", "age", "height", "weight"]
user_index = 0

intents ={
    "intents": [
        {
            "tag": "strength",
            "patterns": ["I want to build strength", "Strength training", "How to get stronger?"],
            "responses": [
                "1. Focus on compound exercises like squats, deadlifts, bench presses, and pull-ups.\n2. Train with progressive overload by gradually increasing weight, reps, or sets.\n3. Aim for 3-5 sets of 4-6 reps per exercise for strength gains.\n4. Take 2-3 minutes of rest between heavy sets to allow optimal recovery.\n5. Ensure a high-protein diet and sufficient calorie intake to support muscle growth."
            ]
        },
        {
            "tag": "weight_loss",
            "patterns": ["I want to lose weight", "Best way to burn fat?", "How can I get leaner?"],
            "responses": [
                "1. Maintain a calorie deficit of 300-500 kcal per day to encourage fat loss.\n2. Perform a combination of HIIT (20-30 minutes) and strength training 4-5 times a week.\n3. Include whole, nutrient-dense foods like lean proteins, vegetables, and complex carbs.\n4. Hydrate well and ensure 7-9 hours of quality sleep to manage stress and hormonal balance."
            ]
        },
        {
            "tag": "muscle_building",
            "patterns": ["I want to build muscle", "How to gain muscle?", "Muscle growth tips?"],
            "responses": [
                "1. Follow a hypertrophy-focused training program with 3-4 sets of 8-12 reps per exercise.\n2. Consume 1.6-2.2g of protein per kg of body weight to optimize muscle recovery and growth.\n3. Ensure a calorie surplus of 200-300 kcal per day with a balanced intake of carbs, proteins, and fats.\n4. Train each muscle group at least twice a week to maximize muscle stimulation."
            ]
        },
        {
            "tag": "diet",
            "patterns": ["Best diet for weight loss?", "What should I eat to gain muscle?", "Healthy eating tips?"],
            "responses": [
                "1. For weight loss, focus on whole foods, lean protein, and a calorie deficit while avoiding processed foods.\n2. For muscle gain, eat protein-rich foods like chicken, fish, eggs, lentils, and pair them with complex carbs like brown rice and oats.\n3. Healthy fats from nuts, seeds, and olive oil are essential for hormone production and overall health.\n4. Maintain proper meal timing and hydration to support metabolism and energy levels."
            ]
        },
        {
            "tag": "cardio",
            "patterns": ["Best cardio workout?", "How much cardio should I do?", "Cardio for fat loss?"],
            "responses": [
                "1. For general health, do 30-45 minutes of moderate-intensity cardio (jogging, cycling) 3-5 times a week.\n2. For fat loss, incorporate HIIT workouts 2-3 times per week alongside strength training.\n3. Mix steady-state and interval cardio to improve endurance and burn more calories.\n4. Track your heart rate and aim for 60-70% of your max heart rate during moderate cardio."
            ]
        },
        {
            "tag": "recovery",
            "patterns": ["How to recover after workouts?", "Best way to avoid muscle soreness?", "Post-workout recovery tips?"],
            "responses": [
                "1. Prioritize sleep (7-9 hours per night) and hydration for optimal recovery.\n2. Perform post-workout stretching and mobility exercises to reduce muscle tightness.\n3. Engage in active recovery like walking or yoga on rest days to keep blood flow active.\n4. Use foam rolling, cold therapy, and proper nutrition to speed up muscle repair."
            ]
        }
    ]
}

lemmatizer = WordNetLemmatizer()
def preprocess(sentence):
    sentence = sentence.lower()
    tokens = word_tokenize(sentence)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in string.punctuation]
    return tokens

patterns, responses, tags = [], [], []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        responses.append(intent['responses'])
        tags.append(intent['tag'])

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(patterns).toarray()
encoder = LabelEncoder()
y = encoder.fit_transform(tags)

model = keras.Sequential([
    keras.layers.Dense(128, input_dim=X.shape[1], activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(len(set(tags)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=50, batch_size=10, verbose=0)

def chatbot_response(user_input, chat_history):
    global user_index

    if user_index < len(user_keys):
        user_data[user_keys[user_index]] = user_input.strip()
        user_index += 1

        if user_index < len(user_keys):
            response = user_questions[user_index]
        else:
            response = "Thank you! Now you can ask fitness-related questions."
    else:
        user_input_vectorized = vectorizer.transform([user_input]).toarray()
        prediction = model.predict(user_input_vectorized)
        tag = encoder.inverse_transform([np.argmax(prediction)])[0]

        for intent in intents['intents']:
            if intent['tag'] == tag:
                response = f"{random.choice(intent['responses'])}"
                break
        else:
            response = "I'm sorry, I didn't quite understand that. Could you rephrase?"

    chat_history.append([user_input, response])
    return chat_history, ""

with gr.Blocks(theme=gr.themes.Soft(primary_hue="orange", secondary_hue="gray")) as chatbot_ui:
    gr.Markdown("""
        <h1 style='text-align: center; color: orange;'>Responsive Personalized Fitness Chatbot</h1>
        <h3 style='text-align: center; color: orange;'>🏋️‍♂️ Enter your details first, then ask fitness-related questions! 🥗</h3>
    """)

    with gr.Row():
        chat = gr.Chatbot()

    with gr.Row():

        user_input = gr.Textbox(placeholder=user_questions[0], label="💬 Your Message", interactive=True)
        submit_btn = gr.Button("🚀 Submit", variant="primary")

    chat_history = gr.State([])

    submit_btn.click(chatbot_response, inputs=[user_input, chat_history], outputs=[chat, user_input])

chatbot_ui.launch()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
<ipython-input-9-5c6b0ddf4c6a>:129: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat = gr.Chatbot()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://57706a86534e312606.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
